## Initialization
Read global config file, all parameters are supposed to be defined here.

In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
import yaml

config_path = '/app/global_config/pod_data_prep_v0.yaml'
with open(config_path, 'r') as stream:
    configs = yaml.load(stream,  Loader=yaml.FullLoader)
    
print('Loaded config:')
print(yaml.dump(configs))#  matplotlib inline

Loaded config:
cross_fold:
- LOOCV_1
- LOOCV_2
- LOOCV_3
- LOOCV_4
- LOOCV_5
detection_candidate_fusion:
  resultpath: /detection_eval/pod/raw
detection_config: detection_itsc_2022
detection_system:
  CenterPoint_Pillar_02: /detection_eval/CenterPoint/Pillar02
  CenterPoint_Voxel_0075: /detection_eval/CenterPoint/Voxel0075
  CenterPoint_Voxel_01: /detection_eval/CenterPoint/Voxel01
  PointPillar: /detection_eval/PointPillar
init:
  dataroot: dataroot
  version: v1.0-trainval



### Init nuscenes dataloader

In [2]:
#  matplotlib inline
from nuscenes import NuScenes

nusc = NuScenes(version=configs['init']['version'], dataroot=configs['init']['dataroot'], verbose=True)

Loading NuScenes tables for version v1.0-trainval...


KeyboardInterrupt: 

### Load prediction boxes
According to global config settings 

--> change to fold wise processing to avoid overloading memory

In [6]:
import os

from nuscenes.eval.detection.data_classes import DetectionBox
from nuscenes.eval.common.data_classes import EvalBoxes
from nuscenes.eval.common.config import config_factory

from nuscenes.utils.srs_utils import load_pred_boxes, load_gt_boxes
cfg = config_factory(configs['detection_config'])



def somethingelse():
    return 0

def something():
    return 0

mode = 'test'

if mode == 'test':
    something()
elif mode == 'else':
    somethingelse()

def f(detector_path,folds):
    pred_boxes = {}
        
    for detector,path in detector_path.items():
        key = detector
        pred_boxes[key] = EvalBoxes()
        
        for fold in folds:
            print(f'Load predicted boxes: {detector} {fold}')

            path_name = os.path.join(path,fold,'results_nusc.json')            
            boxes = load_pred_boxes(nusc = nusc,
                        cfg = cfg,
                        result_path = path_name,
                        box_cls = DetectionBox)
            for sample_token in [s for s in boxes.sample_tokens]:
                pred_boxes[key].add_boxes(sample_token,boxes[sample_token])
        for k,v in pred_boxes.items():
            print(f'{k}: {len(v.sample_tokens)} sample_token')

    return pred_boxes

pred_boxes = f(configs['detection_system'],configs['cross_fold'])

Load predicted boxes: CenterPoint_Pillar_02 LOOCV_1
Load predicted boxes: CenterPoint_Pillar_02 LOOCV_2
Load predicted boxes: CenterPoint_Pillar_02 LOOCV_3
Load predicted boxes: CenterPoint_Pillar_02 LOOCV_4
Load predicted boxes: CenterPoint_Pillar_02 LOOCV_5
CenterPoint_Pillar_02: 34149 sample_token
Load predicted boxes: CenterPoint_Voxel_01 LOOCV_1
Load predicted boxes: CenterPoint_Voxel_01 LOOCV_2
Load predicted boxes: CenterPoint_Voxel_01 LOOCV_3
Load predicted boxes: CenterPoint_Voxel_01 LOOCV_4
Load predicted boxes: CenterPoint_Voxel_01 LOOCV_5
CenterPoint_Pillar_02: 34149 sample_token
CenterPoint_Voxel_01: 34149 sample_token
Load predicted boxes: CenterPoint_Voxel_0075 LOOCV_1
Load predicted boxes: CenterPoint_Voxel_0075 LOOCV_2
Load predicted boxes: CenterPoint_Voxel_0075 LOOCV_3
Load predicted boxes: CenterPoint_Voxel_0075 LOOCV_4
Load predicted boxes: CenterPoint_Voxel_0075 LOOCV_5
CenterPoint_Pillar_02: 34149 sample_token
CenterPoint_Voxel_01: 34149 sample_token
CenterPoint_

In [7]:
gt_boxes = EvalBoxes()
for fold in configs['cross_fold']:
    boxes = load_gt_boxes(  nusc = nusc,
                    cfg = cfg,
                    eval_set = fold + '_val')
    for sample_token in [s for s in boxes.sample_tokens]:
        gt_boxes.add_boxes(sample_token,boxes[sample_token])

print(f'gt_boxes: {len(gt_boxes.sample_tokens)} sample_token')

gt_boxes: 34149 sample_token


### Evaluate predictions
Return optional processparameter as well

In [8]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
from typing import Callable
from nuscenes.eval.common.utils import center_distance, scale_iou, yaw_diff, velocity_l2, attr_acc, cummean

def accumulate(gt_boxes: EvalBoxes,
               pred_boxes: EvalBoxes,
               class_name: str = 'car',
               dist_fcn: Callable = center_distance,
               dist_th: float = 2.0,
               verbose: bool = False):
    """
    Average Precision over predefined different recall thresholds for a single distance threshold.
    The recall/conf thresholds and other raw metrics will be used in secondary metrics.
    :param gt_boxes: Maps every sample_token to a list of its sample_annotations.
    :param pred_boxes: Maps every sample_token to a list of its sample_results.
    :param class_name: Class to compute AP on.
    :param dist_fcn: Distance function used to match detections and ground truths.
    :param dist_th: Distance threshold for a match.
    :param verbose: If true, print debug messages.
    :return: (average_prec, metrics). The average precision value and raw data for a number of metrics.
    """
    # ---------------------------------------------
    # Organize input and initialize accumulators.
    # ---------------------------------------------

    # Count the positives.
    npos = len([1 for gt_box in gt_boxes.all if gt_box.detection_name == class_name])
    #ego_dist = [gt_box.ego_dist for gt_box in gt_boxes.all if gt_box.detection_name == class_name])
    if verbose:
        print("Found {} GT of class {} out of {} total across {} samples.".
              format(npos, class_name, len(gt_boxes.all), len(gt_boxes.sample_tokens)))

    # For missing classes in the GT, return a data structure corresponding to no predictions.
    if npos == 0:
        return None
        # return DetectionMetricData.no_predictions()

    # Organize the predictions in a single list.
    pred_boxes_list = [box for box in pred_boxes.all if box.detection_name == class_name]
    pred_confs = [box.detection_score for box in pred_boxes_list]

    if verbose:
        print("Found {} PRED of class {} out of {} total across {} samples.".
              format(len(pred_confs), class_name, len(pred_boxes.all), len(pred_boxes.sample_tokens)))

    # Sort by confidence.
    sortind = [i for (v, i) in sorted((v, i) for (i, v) in enumerate(pred_confs))][::-1]

    # Do the actual matching.
    tp = []  # Accumulator of true positives.
    fp = []  # Accumulator of false positives.
    fn = []  # Accumulator of false negatives.
    conf = []  # Accumulator of confidences.
    dist = []  # Accumulator of distances.

    # ---------------------------------------------
    # Match and accumulate match data.
    # ---------------------------------------------
    match_data = pd.DataFrame()
    taken = set()  # Initially no gt bounding box is matched.
    for ind in tqdm(sortind):
        pred_box = pred_boxes_list[ind]
        min_dist = np.inf
        match_gt_idx = None

        for gt_idx, gt_box in enumerate(gt_boxes[pred_box.sample_token]):
            # Find closest match among ground truth boxes
            if gt_box.detection_name == class_name and not (pred_box.sample_token, gt_idx) in taken:
                this_distance = dist_fcn(gt_box, pred_box)
                if this_distance < min_dist:
                    min_dist = this_distance
                    match_gt_idx = gt_idx
                    # add reference to closest annotation
                    sample_annotation_token = gt_box.sample_annotation_token
        # If the closest match is close enough according to threshold we have a match!
        is_match = min_dist < dist_th

        if is_match:
            # No match. Mark this as a true positive.
            taken.add((pred_box.sample_token, match_gt_idx))

            tp.append(1)
            fp.append(0)
            fn.append(0)
            conf.append(pred_box.detection_score)
            dist.append(pred_box.ego_dist)

            # add sample token as reference to gt
            pred_box.sample_annotation_token = sample_annotation_token
        else:
            # No match. Mark this as a false positive.
            tp.append(0)
            fp.append(1)
            fn.append(0)
            conf.append(pred_box.detection_score)
            dist.append(pred_box.ego_dist)
            
            # add sample token as reference to gt
            pred_box.sample_annotation_token = None

    # Add missed gt annotations as fn
    for sample_token in gt_boxes.sample_tokens:
        for gt_idx, gt_box in enumerate(gt_boxes[sample_token]):
            if gt_box.detection_name == class_name and not (sample_token, gt_idx) in taken:
                tp.append(0)
                fp.append(0)
                fn.append(1)
                conf.append(-1)
                dist.append(gt_box.ego_dist)
                
    return tp,fp,fn,conf,dist
class_names =  ["car",
                "truck",
                "bus",
                "trailer",
                "construction_vehicle",
                "pedestrian",
                "motorcycle",
                "bicycle",
                "traffic_cone",
                "barrier"]

destination = configs['detection_candidate_fusion']['resultpath']
for k,v in pred_boxes.items():
    print(f'k: {k}, v: {v}')
    for class_name in ['car','pedestrian']:# class_names: #
        data = {}
        data['tp'],data['fp'],data['fn'],data['score'],data['dist'] = accumulate( gt_boxes = gt_boxes, pred_boxes = v, class_name = class_name)

        # safe results
        directory = os.path.join(destination,k,class_name)
        if not os.path.exists(directory):
            os.makedirs(directory)
        result_path = os.path.join(directory,'results.json')
        print(result_path)
        with open(result_path, 'w') as f:
            json.dump(data, f)


k: CenterPoint_Pillar_02, v: EvalBoxes with 2978160 boxes across 34149 samples


100%|██████████| 519547/519547 [01:32<00:00, 5602.56it/s] 


/detection_eval/pod/raw/CenterPoint_Pillar_02/car/results.json


100%|██████████| 210891/210891 [00:14<00:00, 14862.49it/s]


/detection_eval/pod/raw/CenterPoint_Pillar_02/truck/results.json


...

/detection_eval/pod/raw/PointPillar/bicycle/results.json


100%|██████████| 966672/966672 [00:45<00:00, 21284.15it/s]


/detection_eval/pod/raw/PointPillar/traffic_cone/results.json


100%|██████████| 669279/669279 [00:46<00:00, 14277.88it/s]


/detection_eval/pod/raw/PointPillar/barrier/results.json
